# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

In [5]:
import pandas as pd 
df_marketing = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv")
df_marketing.head(3)

df_low = df_marketing [(df_marketing["Total Claim Amount"] < 1000) & (df_marketing["Response"] == "Yes")].reset_index()
df_low

,index,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
1,8,8,FM55990,California,5989.773931,Yes,Premium,College,1/19/11,Employed,...,0.0,1,Personal Auto,Personal L1,Offer2,Branch,739.200000,Sports Car,Medsize,NaN
2,15,15,CW49887,California,4626.801093,Yes,Basic,Master,1/16/11,Employed,...,0.0,1,Special Auto,Special L1,Offer2,Branch,547.200000,SUV,Medsize,NaN
3,19,19,NJ54277,California,3746.751625,Yes,Extended,College,2/26/11,Employed,...,1.0,1,Personal Auto,Personal L2,Offer2,Call Center,19.575683,Two-Door Car,Large,A
4,27,27,MQ68407,Oregon,4376.363592,Yes,Premium,Bachelor,2/28/11,Employed,...,0.0,1,Personal Auto,Personal L3,Offer2,Agent,60.036683,Four-Door Car,Medsize,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394,10844,10844,FM31768,Arizona,5979.724161,Yes,Extended,High School or Below,2/7/11,Employed,...,0.0,3,Personal Auto,Personal L1,Offer2,Agent,547.200000,Four-Door Car,Medsize,NaN
1395,10852,10852,KZ80424,Washington,8382.478392,Yes,Basic,Bachelor,1/27/11,Employed,...,0.0,2,Personal Auto,Personal L2,Offer2,Call Center,791.878042,NaN,NaN,A
1396,10872,10872,XT67997,California,5979.724161,Yes,Extended,High School or Below,2/7/11,Employed,...,0.0,3,Personal Auto,Personal L3,Offer2,Agent,547.200000,Four-Door Car,Medsize,NaN
1397,10887,10887,BY78730,Oregon,8879.790017,Yes,Basic,High School or Below,2/3/11,Employed,...,0.0,7,Special Auto,Special L2,Offer1,Agent,528.200860,SUV,Small,A


2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

In [18]:
df_yes = df_marketing[df_marketing["Response"] == "Yes"]
df_premium = df_yes[df_yes["Coverage"] == "Premium"]

mean_claim_premium = df_premium["Total Claim Amount"].mean().round(2)

df_coverage = df_yes.groupby("Coverage")["Total Claim Amount"].mean().round(2)
display (df_coverage)
print()
avg_claims = df_premium.groupby(["Policy Type", "Gender"])["Total Claim Amount"].mean().round(2)
display (avg_claims)

Coverage
Basic       381.99
Extended    518.90
Premium     652.57
Name: Total Claim Amount, dtype: float64

Policy Type     Gender
Corporate Auto  F          585.32
                M          700.19
Personal Auto   F          665.21
                M          636.90
Special Auto    F          625.37
                M         1027.00
Name: Total Claim Amount, dtype: float64

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [22]:
customers_per_state = df_marketing.groupby("State")["Customer"].nunique()

states_above_500 = customers_per_state[customers_per_state > 500]

states_above_500_df = states_above_500.reset_index(name="number_of_customers")
states_above_500_df

,State,number_of_customers
0,Arizona,1703
1,California,3150
2,Nevada,882
3,Oregon,2601
4,Washington,798


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [27]:
clv_stats = df_marketing.groupby(['Education', 'Gender'])['Customer Lifetime Value'].agg(['max', 'min', 'median']).round(2).reset_index()
clv_stats

,Education,Gender,max,min,median
0,Bachelor,F,73225.96,1904.00,5640.51
1,Bachelor,M,67907.27,1898.01,5548.03
2,College,F,61850.19,1898.68,5623.61
3,College,M,61134.68,1918.12,6005.85
4,Doctor,F,44856.11,2395.57,5332.46
5,Doctor,M,32677.34,2267.60,5577.67
6,High School or Below,F,55277.45,2144.92,6039.55
7,High School or Below,M,83325.38,1940.98,6286.73
8,Master,F,51016.07,2417.78,5729.86
9,Master,M,50568.26,2272.31,5579.10


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here